In [1]:
"""
In this file, we will let a human play a game of Lean (using modal).
"""

import json
import os
import time
from typing import Optional
import pexpect
from vllm import LLM, SamplingParams

from src.games.leaner_lean_game import LeanGame, LeanGameState


In [2]:
# set "VLLM_LOGGING_LEVEL" to "WARNING" to suppress logging
os.environ["VLLM_LOGGING_LEVEL"] = "WARNING"

In [3]:

informal_prefix = r'''/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
'''
formal_statement = r'''theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
    (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
'''
PROBLEM_STATEMENT = informal_prefix + formal_statement
tactic_state = r'''/- tactic state:

a r : ℝ
u : ℕ → ℝ
h₀ : ∀ (k : ℕ), u k = a * r ^ k
h₁ : u 1 = 2
h₂ : u 3 = 6
⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)
-/
'''

llm = LLM(model="deepseek-ai/DeepSeek-Prover-V1.5-RL",
          max_num_batched_tokens=8192,
          trust_remote_code=True,
          dtype="float16",
          tensor_parallel_size=4)

# add a custom stopping token
# which stops on newlines
sampling_params = SamplingParams(
    max_tokens=4096,
    temperature=0.0,
    top_k=1,
    top_p=1.0,
    stop=["\n"]
)


# useful copy+paste for the game.
"""
  -- First, we want to re-write the condition about the second
  -- and fourth terms of the geometric sequence using the definition of a geometric sequence
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  -- Now we can divide the two equations to eliminate $a$ and determine $r$
  have h₃ : r ^ 2 = 3 := by
    nlinarith
  -- Finally, we can substitute back to find $a$
  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
  simpa [h₀] using h₄
"""


WARNING 09-27 18:52:27 config.py:1651] Casting torch.bfloat16 to torch.float16.


INFO 09-27 18:52:27 config.py:890] Defaulting to use mp for distributed inference
INFO 09-27 18:52:27 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='deepseek-ai/DeepSeek-Prover-V1.5-RL', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-Prover-V1.5-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=deepseek-ai/DeepSeek-P

(VllmWorkerProcess pid=35864) /opt/conda/envs/pytorch/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
(VllmWorkerProcess pid=35864)   @torch.library.impl_abstract("xformers_flash::flash_fwd")
(VllmWorkerProcess pid=35863) (VllmWorkerProcess pid=35865) /opt/conda/envs/pytorch/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
/opt/conda/envs/pytorch/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future versio

(VllmWorkerProcess pid=35864) INFO 09-27 18:52:29 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=35865) INFO 09-27 18:52:29 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=35863) INFO 09-27 18:52:29 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
INFO 09-27 18:52:30 utils.py:977] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=35864) INFO 09-27 18:52:30 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=35863) (VllmWorkerProcess pid=35865) INFO 09-27 18:52:30 utils.py:977] Found nccl from library libnccl.so.2
INFO 09-27 18:52:30 utils.py:977] Found nccl from library libnccl.so.2
INFO 09-27 18:52:30 utils.py:977] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=35864) (VllmWorkerProcess pid=35863) INFO 09-27 18:52:30 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 09-27 18:52:30 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=35865) INFO 09-27 18:52:30 pyn

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-27 18:52:34 model_runner.py:926] Loading model weights took 3.2632 GB
(VllmWorkerProcess pid=35865) INFO 09-27 18:52:34 model_runner.py:926] Loading model weights took 3.2632 GB
(VllmWorkerProcess pid=35864) INFO 09-27 18:52:35 model_runner.py:926] Loading model weights took 3.2632 GB
(VllmWorkerProcess pid=35863) INFO 09-27 18:52:35 model_runner.py:926] Loading model weights took 3.2632 GB
INFO 09-27 18:52:36 distributed_gpu_executor.py:57] # GPU blocks: 4848, # CPU blocks: 2184
INFO 09-27 18:52:42 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=35865) INFO 09-27 18:52:42 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_

"\n  -- First, we want to re-write the condition about the second\n  -- and fourth terms of the geometric sequence using the definition of a geometric sequence\n  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,\n    Nat.succ_add]\n  have h₁' : a * r = 2 := by simpa [h₀] using h₁\n  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂\n  -- Now we can divide the two equations to eliminate $a$ and determine $r$\n  have h₃ : r ^ 2 = 3 := by\n    nlinarith\n  -- Finally, we can substitute back to find $a$\n  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by\n    apply eq_or_eq_neg_of_sq_eq_sq <;>\n    field_simp <;>\n    nlinarith\n  simpa [h₀] using h₄\n"

In [4]:
def send_code_read_json(cmd, timeout_start=30, timeout_finish=30, _child : Optional[pexpect.spawn] = None, kill = False):
    if _child is None:
        child = pexpect.spawn(
            f"{DEFAULT_LAKE_PATH} exe repl",
            cwd=DEFAULT_LEAN_WORKSPACE)
    else:
        child = _child

    cmd_json = json.dumps(cmd)
    # print(cmd_json)
    child.send(cmd_json + "\r\n")
    # Read the input itself.
    # This should be printed instantly, so timeout is set to 1 second.
    child.expect_exact(cmd_json + "\r\n", timeout=20)
    assert child.after.decode('utf-8') == cmd_json + "\r\n"
    # print("Sent code to Lean4 REPL.")

    # Read the output.
    # This code is critical; the repl seems to print out some
    # strange non-json stuff before the actual json output,
    # including characters that delete the previous input,
    # such that it doesn't show up in debug output.
    child.expect_exact("{", timeout=timeout_start)
    res = "{"
    print("Received start of output from Lean4 REPL.")
    # while res is not a valid json string, read lines.
    # All of the lines should print essentially instantly,
    # so there are no timeouts in this loop.
    start_time = time.time()
    while True:
        res = res + child.readline().decode('utf-8')
        try:
            # print all chars in res
            json.loads(res.strip())
            break
        except json.JSONDecodeError as e:
            # print(e)
            pass
        if time.time() - start_time > timeout_finish:
            raise TimeoutError("Lean4 REPL timed out.")
        # time.sleep(0.1)

    # kill
    if kill:
        child.close()
    return json.loads(res)

def setup_repl():
    child = pexpect.spawn(
        f"{DEFAULT_LAKE_PATH} exe repl",
        cwd=DEFAULT_LEAN_WORKSPACE)
    send_code_read_json(
        {
            "cmd": LEAN4_DEFAULT_HEADER,
            "allTactics": True,
            "tactics": True,
        },
        _child=child
    )
    return child


HOME_DIR = os.path.expanduser('~')
DEFAULT_LAKE_PATH = f'{HOME_DIR}/.elan/bin/lake'
DEFAULT_LEAN_WORKSPACE = '../mathlib4/'

LEAN4_DEFAULT_HEADER = "import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n"
child = setup_repl()


{"cmd": "import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n", "allTactics": true, "tactics": true}
Sent code to Lean4 REPL.
Received start of output from Lean4 REPL.


In [5]:

comments = None
with open("../src/sample-data/comments.txt", 'r') as file:
    comments = [line.strip() for line in file.readlines()]

game: LeanGame = LeanGame(
    comment_seeds=comments,
)
state: LeanGameState = game.start_state(
    problem=PROBLEM_STATEMENT,
    tactic_state=tactic_state
)


In [6]:
children = []
while not game.is_terminal(state):
    children.append(state)
    # print("Player Must Act".center(80, "#"))
    # print(state.human_printout())
    # print each possible comment with the index in front.
    # for i, comment in enumerate(comments):
    #     print(f"{i}: {comment}")
    # action = int(input("Enter your action: "))
    action = 0
    # action = comments[action]
    # action = input("Enter your action: ")
    state = game.next_state(state, action)

    # print("LLM Must Act".center(80, "#"))
    # print(state.human_printout())
    input_data = state.pre_LLM_rollout()
    # print(input_data)
    outputs = llm.generate(
        input_data,
        sampling_params=sampling_params
    )
    outputs = outputs[0].outputs[0].text + "\n"

    # print(outputs)

    state.post_LLM_rollout(outputs)

    # print("Lean Verifier Must Act!".center(80, "#"))
    # print(state.human_printout())
    # print("Lean4 Input".center(80, "-"))
    lean4_input = state.pre_process()

    lean4_output = send_code_read_json({
        "cmd": lean4_input,
        "allTactics": True,
        "tactics": True,
        "env" : 0
    }, _child=child)

    # print("Lean4 Output".center(80, "-"))
    # print(str(lean4_output.get('messages'))[:100])
    # print(f"({len(str(lean4_output.get('messages')))} characters)")
    state.post_process(lean4_output)


print("Terminated!")
print(state.human_printout())

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.67it/s, est. speed input: 815.65 toks/s, output: 98.94 toks/s]


{"cmd": "/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\frac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : \u211d) (u : \u2115 \u2192 \u211d) (h\u2080 : \u2200 k, u k = a * r ^ k) (h\u2081 : u 1 = 2)\n    (h\u2082 : u 3 = 6) : u 0 = 2 / Real.sqrt 3 \u2228 u 0 = -(2 / Real.sqrt 3) := by\n  simp_all only [h\u2080, Nat.cast_one, Nat.cast_zero, Nat.cast_succ, Nat.cast_zero]\n", "allTactics": true, "tactics": true, "env": 0}
Sent code to Lean4 REPL.
Received start of output from Lean4 REPL.


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  5.18it/s, est. speed input: 1793.67 toks/s, output: 88.63 toks/s]


{"cmd": "/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\frac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : \u211d) (u : \u2115 \u2192 \u211d) (h\u2080 : \u2200 k, u k = a * r ^ k) (h\u2081 : u 1 = 2)\n    (h\u2082 : u 3 = 6) : u 0 = 2 / Real.sqrt 3 \u2228 u 0 = -(2 / Real.sqrt 3) := by\n  simp_all only [h\u2080, Nat.cast_one, Nat.cast_zero, Nat.cast_succ, Nat.cast_zero]\n  have h\u2083 : a * r = 2 := by linarith\n", "allTactics": true, "tactics": true, "env": 0}
Sent code to Lean4 REPL.
Received start of output from Lean4 REPL.


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.54it/s, est. speed input: 1692.36 toks/s, output: 91.22 toks/s]


...

{"cmd": "/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\frac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : \u211d) (u : \u2115 \u2192 \u211d) (h\u2080 : \u2200 k, u k = a * r ^ k) (h\u2081 : u 1 = 2)\n    (h\u2082 : u 3 = 6) : u 0 = 2 / Real.sqrt 3 \u2228 u 0 = -(2 / Real.sqrt 3) := by\n  simp_all only [h\u2080, Nat.cast_one, Nat.cast_zero, Nat.cast_succ, Nat.cast_zero]\n  have h\u2083 : a * r = 2 := by linarith\n  have h\u2084 : a * r ^ 3 = 6 := by linarith\n  have h\u2085 : r ^ 2 = 3 := by\n    nlinarith\n  have h\u2086 : a = 2 / Real.sqrt 3 \u2228 a = -(2 / Real.sqrt 3) := by\n    apply eq_or_eq_neg_of_sq_eq_sq <;> field_simp <;>\n    nlinarith\n", "allTactics": true, "tactics": true, "env": 0}
Sent code to Lean4 REPL.


Received start of output from Lean4 REPL.


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  5.25it/s, est. speed input: 2785.48 toks/s, output: 84.40 toks/s]

{"cmd": "/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\frac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : \u211d) (u : \u2115 \u2192 \u211d) (h\u2080 : \u2200 k, u k = a * r ^ k) (h\u2081 : u 1 = 2)\n    (h\u2082 : u 3 = 6) : u 0 = 2 / Real.sqrt 3 \u2228 u 0 = -(2 / Real.sqrt 3) := by\n  simp_all only [h\u2080, Nat.cast_one, Nat.cast_zero, Nat.cast_succ, Nat.cast_zero]\n  have h\u2083 : a * r = 2 := by linarith\n  have h\u2084 : a * r ^ 3 = 6 := by linarith\n  have h\u2085 : r ^ 2 = 3 := by\n    nlinarith\n  have h\u2086 : a = 2 / Real.sqrt 3 \u2228 a = -(2 / Real.sqrt 3) := by\n    apply eq_or_eq_neg_of_sq_eq_sq <;> field_simp <;>\n    nlinarith\n  simp_all only [pow_zero, mul_one]\n", "allTactics": true, "tactics": true, "env": 0}


Sent code to Lean4 REPL.
Received start of output from Lean4 REPL.
Terminated!
-------------------------------------Status-------------------------------------
Fully processed
-------------------------------------Header-------------------------------------
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

------------------------------------Problem-------------------------------------
/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
    (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
------------------------------------Old Code------------------------------------
  simp_all only [h₀, Nat.cast_one, Nat.cast_zero, Nat.cast_succ, Nat.cast_zero]
  have h₃ : a * r = 2 := by linarith
  have h₄ : a * r ^ 3 = 6 := by linarit